In [1]:
# notebooks/week10_langchain.ipynb

"""
# Week 10: LangChain Integration
## Healthcare Appointment Assistant

### Learning Objectives
1. Understand LangChain Expression Language (LCEL)
2. Build chains for healthcare workflows
3. Create tools that call your ML API
4. Implement conversation memory
5. Build and deploy an agent

### Setup
"""

# Cell 1: Setup and Imports
import os
import sys
from pathlib import Path
from datetime import datetime
from dotenv import load_dotenv

# Add project root
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

load_dotenv()

# Verify setup
print("OpenAI API Key:", "✅" if os.getenv("OPENAI_API_KEY") else "❌")
print("Anthropic API Key:", "✅" if os.getenv("ANTHROPIC_API_KEY") else "❌")

# LangChain imports
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


# Cell 2: Initialize LangChain Config
"""
## Part 1: LangChain Fundamentals

Let's start with basic LangChain concepts.
"""

from src.llm.langchain_config import get_langchain_settings, get_chat_model

settings = get_langchain_settings()
print(f"Default Model: {settings.default_model}")
print(f"Tracing Enabled: {settings.tracing_enabled}")

# Create a model
model = get_chat_model()
print(f"\nModel Type: {type(model).__name__}")


# Cell 3: Simple Chain with LCEL
"""
### LangChain Expression Language (LCEL)

LCEL uses the pipe operator (|) to chain components.
"""

# Simple prompt -> model -> output chain
prompt = ChatPromptTemplate.from_template(
    "Explain why a {risk_level} risk patient might miss their appointment. "
    "Give 3 reasons in bullet points."
)

# Build chain with LCEL
chain = prompt | model | StrOutputParser()

# Test it
result = chain.invoke({"risk_level": "high"})
print("Simple Chain Output:")
print(result)


# Cell 4: Using Our Custom Chains
"""
## Part 2: Healthcare Chains

Using our pre-built chains for healthcare tasks.
"""

from src.llm.chains import RiskExplanationChain, InterventionChain

# Create explanation chain
explanation_chain = RiskExplanationChain()

# Explain a prediction
explanation = explanation_chain.explain(
    probability=0.72,
    risk_tier="HIGH",
    patient_data={
        "age": 35,
        "gender": "F",
        "lead_days": 14,
        "sms_received": 0,
        "appointment_weekday": "Monday",
        "hypertension": 0,
        "diabetes": 1,
        "scholarship": 1
    },
    patient_history={
        "total_appointments": 5,
        "noshow_rate": 0.20
    }
)

print("Prediction Explanation:")
print("=" * 50)
print(explanation)


# Cell 5: Intervention Chain
"""
### Intervention Recommendations
"""

intervention_chain = InterventionChain()

recommendation = intervention_chain.recommend(
    probability=0.78,
    risk_tier="HIGH",
    patient_data={
        "age": 28,
        "lead_days": 21,
        "is_first_appointment": False,
        "previous_noshows": 2,
        "sms_received": 1,
        "neighbourhood": "CENTRO"
    },
    constraints={
        "staff_availability": "Limited (2 staff members today)",
        "phone_slots": 5
    }
)

print("Intervention Recommendation:")
print("=" * 50)
print(recommendation)


# Cell 6: Using Tools
"""
## Part 3: Tools

Tools allow the LLM to take actions, like calling your prediction API.
"""

from src.llm.tools import PredictionTool

# Create the prediction tool
prediction_tool = PredictionTool(api_base_url="http://localhost:8000")

print("Tool Name:", prediction_tool.name)
print("Tool Description:", prediction_tool.description[:200])

# Test the tool (requires API to be running)
# Uncomment if your API is running:
# result = prediction_tool._run(age=45, gender="M", lead_days=10, sms_received=1)
# print("\nTool Result:")
# print(result)


# Cell 7: Conversation Memory
"""
## Part 4: Conversation Memory

Memory allows multi-turn conversations with context.
"""

from src.llm.memory import ConversationMemoryManager

# Create memory manager
memory_manager = ConversationMemoryManager(
    memory_type="buffer_window",
    max_messages=10
)

# Create a session
session_id = memory_manager.create_session()
print(f"Created session: {session_id}")

# Add some conversation turns
memory_manager.add_exchange(
    session_id,
    user_message="What makes a patient high risk?",
    ai_message="High risk patients typically have: long lead times, history of no-shows, and no SMS reminders enabled."
)

memory_manager.add_exchange(
    session_id,
    user_message="How should I contact them?",
    ai_message="For high-risk patients, I recommend a phone call 48-72 hours before the appointment."
)

# View history
history = memory_manager.get_conversation_messages(session_id)
print(f"\nConversation History ({len(history)} messages):")
for msg in history:
    role = "User" if msg.type == "human" else "AI"
    print(f"  {role}: {msg.content[:60]}...")

# Get stats
print("\nMemory Stats:", memory_manager.get_stats())


# Cell 8: Conversation Chain with Memory
"""
### Conversation Chain

Chain that maintains context across messages.
"""

from src.llm.chains import ConversationChain

conv_chain = ConversationChain()

# Create session
session = conv_chain.create_session()

# Multi-turn conversation
responses = []

messages = [
    "What is a no-show in healthcare?",
    "Why is it a problem?",
    "What can we do about high-risk patients?"
]

print("Multi-turn Conversation:")
print("=" * 50)

for msg in messages:
    response = conv_chain.chat(session, msg)
    responses.append(response)
    print(f"\nUser: {msg}")
    print(f"Assistant: {response[:200]}...")

# View full history
history = conv_chain.get_history(session)
print(f"\n\nTotal exchanges: {len(history) // 2}")


# Cell 9: Healthcare Agent
"""
## Part 5: Agents

Agents can autonomously decide when to use tools.
"""

from src.llm.agents import HealthcareAgent, create_healthcare_agent

# Create agent (with verbose mode to see tool calls)
agent = create_healthcare_agent(verbose=True)

# Create session
agent_session = agent.create_session()

print("Agent initialized with tools:", [t.name for t in agent.tools])


# Cell 10: Agent Interaction (Requires Running API)
"""
### Agent Conversation

The agent will decide when to call the prediction tool.
"""

# Note: This requires your prediction API to be running at localhost:8000

# Test messages (some will trigger tool use, some won't)
test_messages = [
    "What's a no-show?",  # General question - no tool needed
    "A 55-year-old male patient has an appointment in 3 weeks. He's missed 2 appointments before. What's his risk?",  # Will use prediction tool
    "What should we do for high-risk patients?",  # General advice
]

print("Agent Interactions:")
print("=" * 50)

for msg in test_messages:
    print(f"\n📝 User: {msg}\n")
    
    try:
        result = agent.chat(agent_session, msg)
        print(f"🤖 Agent: {result['output'][:300]}...")
        
        if result.get('tool_calls'):
            print(f"\n   🔧 Tools used: {[tc['tool'] for tc in result['tool_calls']]}")
    
    except Exception as e:
        print(f"   ⚠️ Error (API might not be running): {e}")

# Agent stats
print("\n\nAgent Stats:", agent.get_stats())


# Cell 11: Chain Orchestrator
"""
## Part 6: Chain Orchestration

The orchestrator routes requests to the appropriate chain.
"""

from src.llm.chains.orchestrator import HealthcareOrchestrator, IntentType

orchestrator = HealthcareOrchestrator()

# Test intent classification
test_queries = [
    "What's the risk for a 40-year-old patient?",  # PREDICT
    "Explain why lead time matters",  # EXPLAIN
    "How should I contact high-risk patients?",  # INTERVENE
    "What's the cancellation policy?",  # POLICY
    "Hello, how are you?",  # GENERAL
]

print("Intent Classification:")
print("=" * 50)

for query in test_queries:
    intent = orchestrator.classify_intent(query)
    print(f"'{query[:40]}...' -> {intent.value}")


# Cell 12: Orchestrator in Action
"""
### Orchestrated Responses
"""

print("\nOrchestrated Responses:")
print("=" * 50)

for query in test_queries[:3]:  # First 3 queries
    print(f"\n📝 Query: {query}")
    
    result = orchestrator.process(
        message=query,
        session_id="test-session"
    )
    
    print(f"🎯 Intent: {result['intent']}")
    print(f"💬 Response: {result['output'][:200]}...")

# Orchestrator stats
print("\n\nOrchestrator Stats:", orchestrator.get_stats())


# Cell 13: Tracing and Metrics
"""
## Part 7: Tracing & Observability
"""

from src.llm.tracing import get_metrics, HealthcareTracingHandler

# Get metrics
metrics = get_metrics()
print("LLM Metrics Summary:")
print(metrics.get_summary())


# Cell 14: Testing the API Endpoints
"""
## Part 8: API Integration

Test the LLM endpoints (requires API to be running).
"""

import httpx

API_BASE = "http://localhost:8000"

# Test chat endpoint
async def test_chat_endpoint():
    async with httpx.AsyncClient() as client:
        response = await client.post(
            f"{API_BASE}/api/v1/llm/chat",
            json={
                "message": "What factors increase no-show risk?",
                "session_id": "test-api-session"
            }
        )
        return response.json()

# Test explanation endpoint
async def test_explain_endpoint():
    async with httpx.AsyncClient() as client:
        response = await client.post(
            f"{API_BASE}/api/v1/llm/explain",
            json={
                "probability": 0.65,
                "risk_tier": "MEDIUM",
                "patient_data": {
                    "age": 42,
                    "gender": "M",
                    "lead_days": 10,
                    "sms_received": 1
                }
            }
        )
        return response.json()

# Uncomment to test (requires running API):
# import asyncio
# result = asyncio.run(test_chat_endpoint())
# print("Chat API Response:", result)


# Cell 15: Exercise - Custom Chain
"""
## Exercises

### Exercise 1: Create a Patient Communication Chain

Create a chain that generates patient-friendly appointment reminders.
The chain should:
- Take appointment details as input
- Generate a warm, professional reminder message
- Include rescheduling instructions
- NOT mention risk predictions

Template to complete:
"""

REMINDER_TEMPLATE = """
Create a friendly appointment reminder for this patient.

Appointment Details:
- Date: {date}
- Time: {time}
- Provider: {provider}
- Location: {location}

Patient Name: {patient_name}

Write a warm, professional reminder that:
1. Confirms the appointment details
2. Tells them what to bring
3. Provides rescheduling instructions (call {phone})
4. Is under 150 words

DO NOT mention any risk assessment or predictions.
"""

# Your code here:
# reminder_prompt = ChatPromptTemplate.from_template(REMINDER_TEMPLATE)
# reminder_chain = reminder_prompt | model | StrOutputParser()
# 
# result = reminder_chain.invoke({
#     "date": "January 25, 2024",
#     "time": "2:30 PM",
#     "provider": "Dr. Smith",
#     "location": "Main Clinic, Room 204",
#     "patient_name": "John",
#     "phone": "555-0123"
# })
# print(result)


# Cell 16: Exercise - Custom Tool
"""
### Exercise 2: Create a Patient History Tool

Create a tool that (simulates) looking up patient history.
This would connect to your database in production.
"""

from langchain_core.tools import tool

@tool
def get_patient_history(patient_id: str) -> str:
    """
    Look up a patient's appointment history.
    
    Args:
        patient_id: The patient's ID number
    
    Returns:
        Patient history summary including past appointments and no-show rate
    """
    # Simulated database lookup
    # In production, this would query your actual database
    mock_data = {
        "P001": {"total": 10, "noshows": 2, "last_noshow": "2023-11-15"},
        "P002": {"total": 5, "noshows": 0, "last_noshow": None},
        "P003": {"total": 8, "noshows": 4, "last_noshow": "2024-01-02"},
    }
    
    if patient_id in mock_data:
        data = mock_data[patient_id]
        rate = data["noshows"] / data["total"] * 100
        return f"""
Patient History for {patient_id}:
- Total Appointments: {data['total']}
- No-Shows: {data['noshows']} ({rate:.1f}%)
- Last No-Show: {data['last_noshow'] or 'Never'}
"""
    else:
        return f"No history found for patient {patient_id}"

# Test the tool
print(get_patient_history("P001"))
print(get_patient_history("P003"))


# Cell 17: Exercise - Enhanced Agent
"""
### Exercise 3: Create an Enhanced Agent

Create an agent with multiple tools:
1. Prediction tool
2. Patient history tool
3. Intervention recommendation tool
"""

# Your code here:
# tools = [
#     PredictionTool(),
#     get_patient_history,
#     # Add more tools
# ]
# 
# enhanced_agent = HealthcareAgent(tools=tools)


# Cell 18: Summary and Deliverables
"""
## Summary

This week you learned:

1. **LCEL (LangChain Expression Language)**
   - Pipe operator for chaining components
   - Prompt | Model | OutputParser pattern

2. **Chains**
   - RiskExplanationChain: Explains predictions
   - InterventionChain: Recommends actions
   - ConversationChain: Multi-turn with memory

3. **Tools**
   - PredictionTool: Calls your ML API
   - Custom tools with @tool decorator

4. **Memory**
   - ConversationMemoryManager
   - Buffer and window memory types
   - Session management

5. **Agents**
   - Autonomous tool selection
   - HealthcareAgent implementation

6. **Orchestration**
   - Intent classification
   - Routing to appropriate chains

## Deliverables

1. ✅ Working explanation chain
2. ✅ Working intervention chain  
3. ✅ Prediction tool integration
4. ✅ Conversation memory
5. ✅ Healthcare agent
6. 📝 Complete exercises 1-3
7. 📝 Test API endpoints

## Next Week: RAG

Week 11 will add:
- Document loading and chunking
- Vector embeddings
- FAISS vector store
- Retrieval-augmented generation
- Policy Q&A with real documents
"""

print("Week 10 Complete! 🎉")
print("\nFinal Metrics:")
print(get_metrics().get_summary())

OpenAI API Key: ✅
Anthropic API Key: ❌


C:\Users\samue\AppData\Roaming\Python\Python314\site-packages\langchain_core\pydantic_v1\__init__.py:15: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1 import *  # noqa: F403


ConfigError: unable to infer type for attribute "artifact"